In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import os, sys, tempfile
import torch.distributed as dist
import torch.optim as optim
import torch.multiprocessing as mp

from torch.utils.data.distributed import DistributedSampler
from apex.parallel import DistributedDataParallel as DDP

In [3]:
transform = transforms.ToTensor()

In [11]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'
os.environ['MASTER_ADDR'] = 'localhost'
os.environ['MASTER_PORT'] = '23097'
dist.init_process_group(backend='nccl', init_method="env://", rank=0, world_size=1) ## init once!

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
train_data = datasets.MNIST(root='../PYTORCH_NOTEBOOKS/Data/', train=True, download=True, transform=transform)

In [5]:
test_data = datasets.MNIST(root='../PYTORCH_NOTEBOOKS/Data/', train=False, download=True, transform=transform)

In [15]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: ../PYTORCH_NOTEBOOKS/Data/
    Split: Train
    StandardTransform
Transform: ToTensor()

In [7]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: ../PYTORCH_NOTEBOOKS/Data/
    Split: Test
    StandardTransform
Transform: ToTensor()

In [18]:
train_loader = DataLoader(
    train_data,
    batch_size= 100,
    shuffle=False,
    sampler= DistributedSampler(train_data),
    num_workers= 6
)
test_loader = DataLoader(
    test_data,
    batch_size= 100,
    shuffle=False,
    sampler= DistributedSampler(test_data),
    num_workers= 6
)

In [ ]:
# 1 COLOR CHANNEL, 6 FILTER (OUTPUT CHANNELS), 3 by 3 KERNEL, STRIDE =1 
conv1 = nn.Conv2d(1,6,3,1) # one grayscale 2D channel, one input channel
conv2 = nn.Conv2d(6,16, 3, 1)